In [112]:
import datasets
from tqdm.notebook import tqdm
from datasets import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

import clip
clip_model, _ = clip.load("ViT-B/32", device)

cuda


In [113]:
import functools

@functools.lru_cache(maxsize=None, typed=False)
def encode_image(img):
    return clip_model.encode_image(img)

class clip_encode_image(object):
    
    def __call__(self, img):
        img = img.unsqueeze(0)
        img =  encode_image(img)
        img = img.squeeze(0)
        return img

    def __repr__(self):
        return self.__class__.__name__+'()'

class Cifar100Clip(Cifar100):
    def __init__(self, num_workers, batch_size, root=None):

        super().__init__(num_workers, batch_size, root)
        
        self.clip_model, self.clip_preprocess = clip.load("ViT-B/32", device)
        self.name = "CIFAR100Clip"
        
        self.transform_fn = self.clip_preprocess
        
#         self.transform_fn.transforms.append(clip_encode_image())
        
        
    def get_train_loaders(self, num_elements_per_class=-1):

        transform_fn = self.transform_fn
        
        train_dataset = CIFAR100(
            root=self.root,
            train=True,
            download=True,
            transform=transform_fn,
        )
#         d = train_dataset.data
#         print(d.shape)
#         d = torch.from_numpy(np.transpose( d, (0, 3, 1, 2)))
#         print(d.shape)
#         d = torch.stack([
#     clip_model.encode_image(x_i.unsqueeze(0)) for i, x_i in enumerate(torch.unbind(d, dim=0), 0)
# ], dim=axis)
#         print(train_dataset.data)
        
#         train_dataset.data = d

        if num_elements_per_class >=0:
            train_dataset = get_n_items_per_class(train_dataset, num_elements_per_class)

        num_train = len(train_dataset)
        indices = list(range(num_train))
        split = int(np.floor(self.valid_size * num_train))

        train_idx = indices[split:]
        train_sampler = SubsetRandomSampler(train_idx)

        train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self.batch_size,
            sampler=train_sampler,
            num_workers=self.num_workers,
        )
        
        # Little hacky - need to improve
        if self.classes == None:
            self.classes = train_dataset.classes
            
        return train_loader


In [114]:
c = Cifar100Clip(6,1)

In [115]:
loaders = c.get_train_loaders()

Files already downloaded and verified


In [116]:
for i,l in loaders:
    print(i.shape, l.shape)
    break

torch.Size([1, 3, 224, 224]) torch.Size([1])


In [117]:
from datasets import *
dataset_obj = Cifar100Clip(6, 75)
train_loader = dataset_obj.get_train_loaders()
# test_loader = dataset_obj.get_test_loader()
classes = dataset_obj.classes

Files already downloaded and verified


In [118]:
import torch.nn as nn
import torch.optim as optim

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
model = LogisticRegression(512,100)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
num_epochs = 500

def num_correct_preds(outputs, labels):
    _, predicted = outputs.max(1)
    return predicted.eq(labels).sum().item()

In [119]:
model.to(device)
model.train()
model = model.to(torch.float16)

for epoch in range(num_epochs):
    
    running_loss = 0.0
    correct = 0.0
    total = 0
    
    for inputs, labels in tqdm(train_loader):
        
        inputs = clip_model.encode_image(inputs.to(device))
        
        inputs = inputs / inputs.norm(dim=-1, keepdim=True)
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()
        running_loss+=loss
        total += len(labels)
        correct += num_correct_preds(outputs, labels)
        
        
    epoch_loss = running_loss/len(train_loader)
    epoch_accuracy = correct*100/total
    print(
        f"Training: Epoch {epoch} || Loss: {epoch_loss:7.3f} || Accuracy: {epoch_accuracy:6.2f}%"
    )
#     scheduler.step()

  0%|          | 0/600 [00:00<?, ?it/s]

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/multimodal/model/multimodal_transformer/___torch_mangle_9591.py", line 19, in encode_image
    _0 = self.visual
    input = torch.to(image, torch.device("cuda:0"), 5, False, False, None)
    return (_0).forward(input, )
            ~~~~~~~~~~~ <--- HERE
  def encode_text(self: __torch__.multimodal.model.multimodal_transformer.___torch_mangle_9591.Multimodal,
    input: Tensor) -> Tensor:
  File "code/__torch__/multimodal/model/multimodal_transformer.py", line 34, in forward
    x2 = torch.add(x1, torch.to(_4, 5, False, False, None), alpha=1)
    x3 = torch.permute((_3).forward(x2, ), [1, 0, 2])
    x4 = torch.permute((_2).forward(x3, ), [1, 0, 2])
                        ~~~~~~~~~~~ <--- HERE
    _15 = torch.slice(x4, 0, 0, 9223372036854775807, 1)
    x5 = torch.slice(torch.select(_15, 1, 0), 1, 0, 9223372036854775807, 1)